In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 1 - Opção de link via google drive

file_id_medico = '1tu0E4Xqt7FeprldGQYI4uypcncIOwDHl' 
url_medico = f'https://drive.google.com/uc?export=download&id={file_id_medico}'

file_id_pacientes = '1d-ZZjoFHCRym5tGei5cE05974uzPIuQT' 
url_pacientes = f'https://drive.google.com/uc?export=download&id={file_id_pacientes}'

file_id_clinicas = '14w0KYUbEJicByKS7M3upVvWzXKLH8kzQ' 
url_clinicas = f'https://drive.google.com/uc?export=download&id={file_id_clinicas}'

file_id_consultas = '12bPlpo8oiaEOIuwQni1lKzc6fWKm7QfI' 
url_consultas = f'https://drive.google.com/uc?export=download&id={file_id_consultas}'

file_id_avaliacoes = '1X5cWJUBa712NGzpR5THEU1T4xlEozfoh' 
url_avaliacoes = f'https://drive.google.com/uc?export=download&id={file_id_avaliacoes}'


# 2 - importando todas as bases
df_medico = pd.read_csv (url_medico,sep=';', encoding='utf-8')
df_pacientes = pd.read_csv (url_pacientes,sep=',', encoding='utf-8')
df_clinicas = pd.read_csv (url_clinicas,sep=',', encoding='utf-8')
df_consultas = pd.read_csv (url_consultas,sep=',', encoding='utf-8')
df_avaliacoes = pd.read_csv (url_avaliacoes,sep=',', encoding='utf-8')


# 3- Criando uma lista pra facilitar o concat
lista_dfs = [df_medico,df_pacientes,df_clinicas,df_consultas,df_avaliacoes]

# 4 - Transformando as colunas de data no tipo data

df_consultas['data_hora_agendada'] = pd.to_datetime(df_consultas['data_hora_agendada'], errors= "coerce", dayfirst=True)
df_consultas['data_hora_inicio'] = pd.to_datetime(df_consultas['data_hora_inicio'], errors= "coerce", dayfirst=True)

# 5 - Cálculo de tempo de espera consultas realizadas_______________________________________________________________________________________________________________

# 5.1 - Filtrando apenas as consultas realizadas 
df_consultas_espera = df_consultas.loc[df_consultas['status'] == 'Realizada'].copy()

# 5.2 - Calculando tempo de espera em minutos
df_consultas_espera['Tempo_de_espera(min)'] = ((df_consultas_espera['data_hora_inicio'] - df_consultas_espera['data_hora_agendada']).dt.total_seconds() / 60)


# 6 - Cálculo de tempo de espera consultas realizadas_______________________________________________________________________________________________________________

# 6.1 - Filtrando apenas as consultas canceladas
df_consultas_canceladas= df_consultas.loc[df_consultas['status'] == 'Cancelada'].copy()
df_consultas_canceladas.to_csv('Consultas_Canceladas.csv', index=False)

# 6.2 - Calculando tempo de espera em minutos
df_consultas_canceladas['Tempo_de_espera(min)'] = ((df_consultas_canceladas['data_hora_inicio'] - df_consultas_canceladas['data_hora_agendada']).dt.total_seconds() / 60)

# 6.3 - Transformando em Float pra facilitar futuros calculos
df_consultas_espera['Tempo_de_espera(min)'] = df_consultas_espera['Tempo_de_espera(min)'].astype('Float64')


C:\Users\36134552025.2\AppData\Local\Temp\ipykernel_15104\5942310.py:36: UserWarning: Parsing dates in %Y-%m-%d %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_consultas['data_hora_agendada'] = pd.to_datetime(df_consultas['data_hora_agendada'], errors= "coerce", dayfirst=True)
C:\Users\36134552025.2\AppData\Local\Temp\ipykernel_15104\5942310.py:37: UserWarning: Parsing dates in %Y-%m-%d %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_consultas['data_hora_inicio'] = pd.to_datetime(df_consultas['data_hora_inicio'], errors= "coerce", dayfirst=True)


In [3]:
# ___________________________________________________Média e mediana por Especialidade_______________________________________________________________________________

# CARDIOLOGIA -----------------------------------------------------------------------------------------
df_cardio = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Cardiologia']

# transformando em array
dados_cardio= np.array(df_cardio['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_cardio = np.mean(df_cardio['Tempo_de_espera(min)'])
mediana_cardio = np.median(df_cardio['Tempo_de_espera(min)'])
distancia_cardio = ((media_cardio - mediana_cardio)/mediana_cardio) *100
desvio_padrao_cardio = np.std(df_cardio['Tempo_de_espera(min)'], ddof=1)

#quartis
cardio_q1= np.percentile(dados_cardio,25)
cardio_q2= np.percentile(dados_cardio,50)
cardio_q3= np.percentile(dados_cardio,75)

# feito pelo desvio padrão 
cardio_iqr= cardio_q3 - cardio_q1
cardio_ls= media_cardio + 2*desvio_padrao_cardio #limite superior
cardio_li= media_cardio - 2*desvio_padrao_cardio # limite inferior


cardio_outliers = df_cardio[(df_cardio['Tempo_de_espera(min)'] < cardio_li) | (df_cardio['Tempo_de_espera(min)'] > cardio_ls)]

# Exibição dos resultados
print(f'Média: {media_cardio:.2f}\nMediana: {mediana_cardio:.2f}\nDistância: {distancia_cardio:.2f}%')
cardio_outliers


Média: 93.61
Mediana: 45.00
Distância: 108.03%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
20,21,72,69,34,Cardiologia,2023-09-29 23:30:00,2023-09-30 04:30:00,Realizada,300.0
86,87,66,15,50,Cardiologia,2023-11-24 11:59:00,2023-11-24 16:59:00,Realizada,300.0
192,193,1,37,93,Cardiologia,2023-04-19 16:49:00,2023-04-19 21:49:00,Realizada,300.0
276,277,56,15,37,Cardiologia,2023-09-04 22:21:00,2023-09-05 03:21:00,Realizada,300.0
332,333,31,54,82,Cardiologia,2023-10-17 01:11:00,2023-10-17 06:11:00,Realizada,300.0
...,...,...,...,...,...,...,...,...,...
4738,4739,29,15,57,Cardiologia,2023-02-28 05:18:00,2023-02-28 10:18:00,Realizada,300.0
4815,4816,60,12,12,Cardiologia,2023-12-12 18:03:00,2023-12-12 23:03:00,Realizada,300.0
4904,4905,85,75,44,Cardiologia,2023-12-13 09:32:00,2023-12-13 14:32:00,Realizada,300.0
4909,4910,68,18,3,Cardiologia,2023-03-29 10:05:00,2023-03-29 15:05:00,Realizada,300.0


In [4]:
# DERMATOLOGIA   -----------------------------------------------------------------------------------------
df_dermato = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Dermatologia']

# transformando em array
dados_dermato= np.array(df_dermato['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_dermato = np.mean(df_dermato['Tempo_de_espera(min)'])
mediana_dermato = np.median(df_dermato['Tempo_de_espera(min)'])
distancia_dermato = ((media_dermato - mediana_dermato)/mediana_dermato) *100
desvio_padrao_dermato = np.std(df_dermato['Tempo_de_espera(min)'], ddof=1)

#quartis
dermato_q1= np.percentile(dados_dermato,25)
dermato_q2= np.percentile(dados_dermato,50)
dermato_q3= np.percentile(dados_dermato,75)

# feito pela formula normal com mudança na regra do iqr
dermato_iqr= dermato_q3 - dermato_q1
dermato_ls= dermato_q3 + (0.5*dermato_iqr) #limite superior
dermato_li= dermato_q1 - (0.5*dermato_iqr) # limite inferior

# feito pelo desvio padrão 
dermato_ls_dp= media_dermato + 2*desvio_padrao_dermato #limite superior
dermato_li_dp= media_dermato - 2*desvio_padrao_dermato # limite inferior

dermato_outliers = df_dermato[(df_dermato['Tempo_de_espera(min)'] < dermato_li) | (df_dermato['Tempo_de_espera(min)'] > dermato_ls)]
dermato_outliers_dp = df_dermato[(df_dermato['Tempo_de_espera(min)'] < dermato_li_dp) | (df_dermato['Tempo_de_espera(min)'] > dermato_ls_dp)]

# Exibição dos resultados
print(f'Média: {media_dermato:.2f}\nMediana: {mediana_dermato:.2f}\nDistância: {distancia_dermato:.2f}%')
dermato_outliers

Média: 98.55
Mediana: 60.00
Distância: 64.25%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
195,196,42,25,90,Dermatologia,2023-03-28 22:42:00,2023-03-29 03:42:00,Realizada,300.0
330,331,28,20,71,Dermatologia,2023-12-09 07:49:00,2023-12-09 12:49:00,Realizada,300.0
675,676,60,7,22,Dermatologia,2023-12-03 11:11:00,2023-12-03 16:11:00,Realizada,300.0
678,679,58,20,5,Dermatologia,2023-08-12 14:08:00,2023-08-12 19:08:00,Realizada,300.0
816,817,21,14,54,Dermatologia,2023-11-28 18:25:00,2023-11-28 23:25:00,Realizada,300.0
845,846,17,14,50,Dermatologia,2023-08-28 09:30:00,2023-08-28 14:30:00,Realizada,300.0
954,955,14,20,55,Dermatologia,2023-08-26 05:33:00,2023-08-26 10:33:00,Realizada,300.0
1011,1012,95,58,11,Dermatologia,2023-05-19 17:50:00,2023-05-19 22:50:00,Realizada,300.0
1061,1062,70,90,63,Dermatologia,2023-03-11 02:17:00,2023-03-11 07:17:00,Realizada,300.0
1111,1112,45,25,42,Dermatologia,2023-05-06 10:30:00,2023-05-06 15:30:00,Realizada,300.0


In [5]:
# GINECOLOGIA --------------------------------------------------------------------------------------------

df_gineco = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Ginecologia']

# transformando em array
dados_gineco= np.array(df_gineco['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_gineco = np.mean(df_gineco['Tempo_de_espera(min)'])
mediana_gineco = np.median(df_gineco['Tempo_de_espera(min)'])
distancia_gineco = ((media_gineco - mediana_gineco)/mediana_gineco) *100
desvio_padrao_gineco = np.std(df_gineco['Tempo_de_espera(min)'], ddof=1)

#quartis
gineco_q1= np.percentile(dados_gineco,25)
gineco_q2= np.percentile(dados_gineco,50)
gineco_q3= np.percentile(dados_gineco,75)

# feito pela formula normal com mudança na regra do iqr
gineco_iqr= gineco_q3 - gineco_q1
gineco_ls= gineco_q3 + (0.5*gineco_iqr) #limite superior
gineco_li= gineco_q1 - (0.5*gineco_iqr) # limite inferior

# feito pelo desvio padrão 
gineco_ls_dp= media_gineco + 2*desvio_padrao_gineco #limite superior
gineco_li_dp= media_gineco - 2*desvio_padrao_gineco # limite inferior

gineco_outliers = df_gineco[(df_gineco['Tempo_de_espera(min)'] < gineco_li) | (df_gineco['Tempo_de_espera(min)'] > gineco_ls)]
gineco_outliers_dp = df_gineco[(df_gineco['Tempo_de_espera(min)'] < gineco_li_dp) | (df_gineco['Tempo_de_espera(min)'] > gineco_ls_dp)]

# Exibição dos resultados
print(f'Média: {media_gineco:.2f}\nMediana: {mediana_gineco:.2f}\nDistância: {distancia_gineco:.2f}%')
gineco_outliers

Média: 95.02
Mediana: 30.00
Distância: 216.72%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
42,43,28,70,17,Ginecologia,2023-09-21 08:45:00,2023-09-21 13:45:00,Realizada,300.0
164,165,1,53,58,Ginecologia,2023-09-08 23:49:00,2023-09-09 04:49:00,Realizada,300.0
241,242,43,32,17,Ginecologia,2023-10-14 12:43:00,2023-10-14 17:43:00,Realizada,300.0
243,244,30,43,100,Ginecologia,2023-02-23 17:20:00,2023-02-23 22:20:00,Realizada,300.0
410,411,34,84,100,Ginecologia,2023-07-30 14:47:00,2023-07-30 19:47:00,Realizada,300.0
448,449,18,62,20,Ginecologia,2023-06-15 15:39:00,2023-06-15 20:39:00,Realizada,300.0
501,502,65,59,48,Ginecologia,2023-01-23 00:13:00,2023-01-23 05:13:00,Realizada,300.0
564,565,95,79,26,Ginecologia,2023-06-13 16:26:00,2023-06-13 21:26:00,Realizada,300.0
596,597,48,59,63,Ginecologia,2023-04-30 03:53:00,2023-04-30 08:53:00,Realizada,300.0
693,694,68,62,1,Ginecologia,2023-05-01 18:44:00,2023-05-01 23:44:00,Realizada,300.0


In [7]:
# NEUROLOGIA ---------------------------------------------------------------------------------------------

df_neuro = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Neurologia']

# transformando em array
dados_neuro= np.array(df_neuro['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_neuro = np.mean(df_neuro['Tempo_de_espera(min)'])
mediana_neuro = np.median(df_neuro['Tempo_de_espera(min)'])
distancia_neuro = ((media_neuro - mediana_neuro)/mediana_neuro) *100
desvio_padrao_neuro = np.std(df_neuro['Tempo_de_espera(min)'], ddof=1)

#quartis
neuro_q1= np.percentile(dados_neuro,25)
neuro_q2= np.percentile(dados_neuro,50)
neuro_q3= np.percentile(dados_neuro,75)

# feito pela formula normal com mudança na regra do iqr
neuro_iqr= neuro_q3 - neuro_q1
neuro_ls= neuro_q3 + (0.5*neuro_iqr) #limite superior
neuro_li= neuro_q1 - (0.5*neuro_iqr) # limite inferior

# feito pelo desvio padrão 
neuro_ls_dp= media_neuro + 2*desvio_padrao_neuro #limite superior
neuro_li_dp= media_neuro - 2*desvio_padrao_neuro # limite inferior

neuro_outliers = df_neuro[(df_neuro['Tempo_de_espera(min)'] < neuro_li) | (df_neuro['Tempo_de_espera(min)'] > neuro_ls)]
neuro_outliers_dp = df_neuro[(df_neuro['Tempo_de_espera(min)'] < neuro_li_dp) | (df_neuro['Tempo_de_espera(min)'] > neuro_ls_dp)]

# Exibição dos resultados
print(f'Média: {media_neuro:.2f}\nMediana: {mediana_neuro:.2f}\nDistância: {distancia_neuro:.2f}%')
neuro_outliers


Média: 95.20
Mediana: 60.00
Distância: 58.66%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
35,36,22,34,68,Neurologia,2023-11-14 15:26:00,2023-11-14 20:26:00,Realizada,300.0
228,229,87,96,87,Neurologia,2023-04-06 10:48:00,2023-04-06 15:48:00,Realizada,300.0
292,293,73,98,75,Neurologia,2023-06-13 10:35:00,2023-06-13 15:35:00,Realizada,300.0
360,361,48,57,98,Neurologia,2023-01-28 23:50:00,2023-01-29 04:50:00,Realizada,300.0
553,554,66,96,67,Neurologia,2023-07-23 03:49:00,2023-07-23 08:49:00,Realizada,300.0
690,691,4,91,90,Neurologia,2023-05-09 01:02:00,2023-05-09 06:02:00,Realizada,300.0
763,764,66,85,48,Neurologia,2023-07-24 10:05:00,2023-07-24 15:05:00,Realizada,300.0
843,844,7,71,56,Neurologia,2023-11-02 09:41:00,2023-11-02 14:41:00,Realizada,300.0
905,906,79,85,40,Neurologia,2023-09-10 11:31:00,2023-09-10 16:31:00,Realizada,300.0
1738,1739,8,57,8,Neurologia,2023-04-20 18:17:00,2023-04-20 23:17:00,Realizada,300.0


In [8]:
# ORTOPEDIA -----------------------------------------------------------------------------------------

df_ortoped = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Ortopedia']

# transformando em array
dados_ortoped= np.array(df_ortoped['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_ortoped = np.mean(df_ortoped['Tempo_de_espera(min)'])
mediana_ortoped = np.median(df_ortoped['Tempo_de_espera(min)'])
distancia_ortoped = ((media_ortoped - mediana_ortoped)/mediana_ortoped) *100
desvio_padrao_ortoped = np.std(df_ortoped['Tempo_de_espera(min)'], ddof=1)

#quartis
ortoped_q1= np.percentile(dados_ortoped,25)
ortoped_q2= np.percentile(dados_ortoped,50)
ortoped_q3= np.percentile(dados_ortoped,75)

# feito pela formula normal com mudança na regra do iqr
ortoped_iqr= ortoped_q3 - ortoped_q1
ortoped_ls= ortoped_q3 + (0.5*ortoped_iqr) #limite superior
ortoped_li= ortoped_q1 - (0.5*ortoped_iqr) # limite inferior

# feito pelo desvio padrão 
ortoped_ls_dp= media_ortoped + 2*desvio_padrao_ortoped #limite superior
ortoped_li_dp= media_ortoped - 2*desvio_padrao_ortoped # limite inferior

ortoped_outliers = df_ortoped[(df_ortoped['Tempo_de_espera(min)'] < ortoped_li) | (df_ortoped['Tempo_de_espera(min)'] > ortoped_ls)]
ortoped_outliers_dp = df_ortoped[(df_ortoped['Tempo_de_espera(min)'] < ortoped_li_dp) | (df_ortoped['Tempo_de_espera(min)'] > ortoped_ls_dp)]

# Exibição dos resultados
print(f'Média: {media_ortoped:.2f}\nMediana: {mediana_ortoped:.2f}\nDistância: {distancia_ortoped:.2f}%')

ortoped_outliers

Média: 94.66
Mediana: 30.00
Distância: 215.52%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
13,14,90,88,83,Ortopedia,2023-01-26 23:55:00,2023-01-27 04:55:00,Realizada,300.0
57,58,31,52,16,Ortopedia,2023-12-09 19:47:00,2023-12-10 00:47:00,Realizada,300.0
159,160,54,78,77,Ortopedia,2023-10-15 21:35:00,2023-10-16 02:35:00,Realizada,300.0
161,162,76,55,40,Ortopedia,2023-07-27 00:45:00,2023-07-27 05:45:00,Realizada,300.0
203,204,98,63,70,Ortopedia,2023-01-14 01:41:00,2023-01-14 06:41:00,Realizada,300.0
...,...,...,...,...,...,...,...,...,...
4692,4693,82,6,98,Ortopedia,2023-09-02 11:11:00,2023-09-02 16:11:00,Realizada,300.0
4768,4769,97,77,51,Ortopedia,2023-04-24 23:31:00,2023-04-25 04:31:00,Realizada,300.0
4882,4883,21,4,27,Ortopedia,2023-09-19 09:39:00,2023-09-19 14:39:00,Realizada,300.0
4886,4887,49,77,23,Ortopedia,2023-01-24 02:57:00,2023-01-24 07:57:00,Realizada,300.0


In [9]:
# PEDIATRIA -----------------------------------------------------------------------------------------

df_pedi = df_consultas_espera.loc[df_consultas_espera['especialidade']== 'Pediatria']

# transformando em array
dados_pedi= np.array(df_pedi['Tempo_de_espera(min)'])

#Calculo de medidas de tendencias centrais 
media_pedi = np.mean(df_pedi['Tempo_de_espera(min)'])
mediana_pedi = np.median(df_pedi['Tempo_de_espera(min)'])
distancia_pedi = ((media_pedi - mediana_pedi)/mediana_pedi) *100
desvio_padrao_pedi = np.std(df_pedi['Tempo_de_espera(min)'], ddof=1)

#quartis
pedi_q1= np.percentile(dados_pedi,25)
pedi_q2= np.percentile(dados_pedi,50)
pedi_q3= np.percentile(dados_pedi,75)

# feito pela formula normal com mudança na regra do iqr
pedi_iqr= pedi_q3 - pedi_q1
pedi_ls= pedi_q3 + (0.5*pedi_iqr) #limite superior
pedi_li= pedi_q1 - (0.5*pedi_iqr) # limite inferior

# feito pelo desvio padrão 
pedi_ls_dp= media_pedi + 2*desvio_padrao_pedi #limite superior
pedi_li_dp= media_pedi - 2*desvio_padrao_pedi # limite inferior

pedi_outliers = df_pedi[(df_pedi['Tempo_de_espera(min)'] < pedi_li) | (df_pedi['Tempo_de_espera(min)'] > pedi_ls)]
pedi_outliers_dp = df_pedi[(df_pedi['Tempo_de_espera(min)'] < pedi_li_dp) | (df_pedi['Tempo_de_espera(min)'] > pedi_ls_dp)]


# Exibição dos resultados
print(f'Média: {media_pedi:.2f}\nMediana: {mediana_pedi:.2f}\nDistância: {distancia_pedi:.2f}%')
pedi_outliers

Média: 98.08
Mediana: 30.00
Distância: 226.94%


,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min)
56,57,66,11,24,Pediatria,2023-01-25 22:44:00,2023-01-26 03:44:00,Realizada,300.0
230,231,80,72,42,Pediatria,2023-11-30 05:37:00,2023-11-30 10:37:00,Realizada,300.0
266,267,55,72,99,Pediatria,2023-10-25 23:31:00,2023-10-26 04:31:00,Realizada,300.0
290,291,19,33,26,Pediatria,2023-03-05 04:03:00,2023-03-05 09:03:00,Realizada,300.0
382,383,4,11,6,Pediatria,2023-12-12 08:31:00,2023-12-12 13:31:00,Realizada,300.0
...,...,...,...,...,...,...,...,...,...
4658,4659,89,66,15,Pediatria,2023-10-11 16:43:00,2023-10-11 21:43:00,Realizada,300.0
4824,4825,15,2,83,Pediatria,2023-12-08 02:22:00,2023-12-08 07:22:00,Realizada,300.0
4930,4931,75,10,17,Pediatria,2023-05-16 17:51:00,2023-05-16 22:51:00,Realizada,300.0
4934,4935,51,10,59,Pediatria,2023-12-12 09:14:00,2023-12-12 14:14:00,Realizada,300.0


In [10]:
# ___________________________________________________Média e mediana por Médico_______________________________________________________________________________

# Pd merge para juntar o df de medico e de tempo
df_medico_merge = pd.merge(df_medico, df_consultas_espera, on='id_medico', how='inner')

# Média, mediana e distância
df_med_media = df_medico_merge.groupby('nome')['Tempo_de_espera(min)'].mean().round(1).reset_index()
df_med_mediana= df_medico_merge.groupby('nome')['Tempo_de_espera(min)'].median().reset_index()

# Adicionando uma coluna "distancia" atravez do calculo feito com a media e mediana 
df_distancia = pd.merge(df_med_media, df_med_mediana, on='nome', suffixes=('_media', '_mediana')) #suffixer: adicionando um sufixo nas colunas novas
df_distancia['distancia'] = (df_distancia['Tempo_de_espera(min)_media'] - df_distancia['Tempo_de_espera(min)_mediana']).abs().round(2) #abs para trazer valor absoluto

# Função para mostrar os dados do médico através de uma busca ***********************************

def buscar_medico(nome):
    resultado = df_distancia.loc[df_distancia['nome'] == nome ]
    print(resultado)

buscar_medico("Dr(a). Ricardo Souza") # medico usado como exemplo


                    nome  Tempo_de_espera(min)_media  \
58  Dr(a). Ricardo Souza                        73.1   

    Tempo_de_espera(min)_mediana  distancia  
58                          30.0       43.1  


In [11]:
# Função que busca os outliers por medico********************************************************

def buscar_outliers_medico(nome):
    # Filtra os dados do médico
    df_filtro_unitario = df_medico_merge.loc[df_medico_merge['nome'] == nome]

    if df_filtro_unitario.empty:
        print(f"Médico '{nome}' não encontrado.")
        return
    else: 
        print(f"Médico '{nome}' encontrado.")

    # criando o array
    dados_med= np.array(df_filtro_unitario['Tempo_de_espera(min)'])

    # Calcula Q1, Q3 e IQR
    q1 = np.percentile(dados_med, 25)
    q3 = np.percentile(dados_med,75)
    iqr = q3 - q1

    limite_inferior = q1 - (1.5 * iqr)
    limite_superior = q3 + (1.5 * iqr)

    # Filtra as linhas que são outliers
    outliers = df_filtro_unitario[(df_filtro_unitario['Tempo_de_espera(min)'] < limite_inferior) | (df_filtro_unitario['Tempo_de_espera(min)'] > limite_superior)]

    # Retorna os resultados
    if outliers.empty:
        print(f"Nenhum outlier encontrado para o médico '{nome}'.")
    else:
        print(f"Outliers encontrados para o médico '{nome}':")
        print(outliers.to_string(index=False))

buscar_outliers_medico("Dr(a). Ricardo Souza") # medico de exmplo para a consulta

Médico 'Dr(a). Ricardo Souza' encontrado.
Outliers encontrados para o médico 'Dr(a). Ricardo Souza':
 id_medico                 nome especialidade_x  id_consulta  id_paciente  id_clinica especialidade_y  data_hora_agendada    data_hora_inicio    status  Tempo_de_espera(min)
         2 Dr(a). Ricardo Souza       Pediatria         1273           33          95       Pediatria 2023-02-10 14:44:00 2023-02-10 19:44:00 Realizada                 300.0
         2 Dr(a). Ricardo Souza       Pediatria         4825           15          83       Pediatria 2023-12-08 02:22:00 2023-12-08 07:22:00 Realizada                 300.0


In [12]:
# ___________________________________________________Média e mediana por Clínica_______________________________________________________________________________
        
# Pd merge para juntar o df de clinica e de tempo
df_clinica_merge = pd.merge(df_clinicas, df_consultas_espera, on='id_clinica', how='inner')

# Média, mediana e distância

df_clinica_media = df_clinica_merge.groupby('nome')['Tempo_de_espera(min)'].mean().round(1).reset_index()
df_clinica_mediana= df_clinica_merge.groupby('nome')['Tempo_de_espera(min)'].median().reset_index()

# Junta média e mediana em um df para criar uma tabela com media, mediana e distancia

df_distancia_clinica= pd.merge(df_clinica_media, df_clinica_mediana, on='nome', suffixes=('_media', '_mediana')) #suffixer para renomear as colunas
df_distancia_clinica['distancia'] = (df_distancia_clinica['Tempo_de_espera(min)_media'] - df_distancia_clinica['Tempo_de_espera(min)_mediana']).abs().round(2) #abs para trazer valor absoluto

# Função para mostrar os dados da clinica
def buscar_clinica(nome):
    resultado = df_distancia_clinica.loc[df_distancia_clinica['nome'] == nome ]
    print(resultado)

buscar_clinica("Bem Viver") #

        nome  Tempo_de_espera(min)_media  Tempo_de_espera(min)_mediana  \
0  Bem Viver                        93.4                          60.0   

   distancia  
0       33.4  


In [14]:
#Função para mostrar os outliers da clinica

def buscar_outliers_clinica(nome):

    # Filtra os dados da clinica
    df_filtro_unitario = df_clinica_merge[df_clinica_merge['nome'] == nome]

    if df_filtro_unitario.empty:
        print(f"Médico '{nome}' não encontrado.")
        return
    
    # criando o array
    dados_clinica= np.array(df_filtro_unitario['Tempo_de_espera(min)'])

    # Q1, Q3 e IQR
    q1 = np.percentile(dados_clinica,25) 
    q3 = np.percentile(dados_clinica,75)
    iqr = q3 - q1

    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    # Filtra as linhas que são outliers
    outliers = df_filtro_unitario[(df_filtro_unitario['Tempo_de_espera(min)'] < limite_inferior) |(df_filtro_unitario['Tempo_de_espera(min)'] > limite_superior)]

    # resultado
    if outliers.empty:
        print(f"Nenhum outlier encontrado para a clinica '{nome}'.")
    else:
        print(f"Outliers encontrados para a clinica '{nome}':")
        print(outliers.to_string(index=False))
    
buscar_outliers_clinica("Bem Viver")

Nenhum outlier encontrado para a clinica 'Bem Viver'.


In [15]:

# Correlação entre tempo de espera e satisfacao
        
df_tempo_espera = df_consultas_espera.groupby('id_consulta')['Tempo_de_espera(min)'].sum().reset_index()
df_tempo_espera = df_consultas_espera.sort_values(by='Tempo_de_espera(min)', ascending= False)
df_nota_satisfacao = df_avaliacoes.groupby('id_consulta')['nota_satisfacao'].sum().reset_index()
df_nota_satisfacao = df_nota_satisfacao.sort_values(by='nota_satisfacao', ascending= True)


#merge inner 
tempo_espera_satisfacao = pd.merge(df_tempo_espera, df_nota_satisfacao, on = 'id_consulta', how= 'inner')
tempo_espera_satisfacao

,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,Tempo_de_espera(min),nota_satisfacao
0,4743,79,71,42,Neurologia,2023-02-09 14:03:00,2023-02-09 19:03:00,Realizada,300.0,5
1,1273,33,2,95,Pediatria,2023-02-10 14:44:00,2023-02-10 19:44:00,Realizada,300.0,3
2,4444,86,79,93,Ginecologia,2023-06-16 16:30:00,2023-06-16 21:30:00,Realizada,300.0,3
3,231,80,72,42,Pediatria,2023-11-30 05:37:00,2023-11-30 10:37:00,Realizada,300.0,2
4,2486,100,81,15,Ginecologia,2023-12-12 02:40:00,2023-12-12 07:40:00,Realizada,300.0,2
...,...,...,...,...,...,...,...,...,...,...
59,4664,1,24,37,Pediatria,2023-07-19 04:01:00,2023-07-19 04:01:00,Realizada,0.0,2
60,1351,35,26,43,Cardiologia,2023-02-16 07:35:00,2023-02-16 07:35:00,Realizada,0.0,4
61,1393,48,24,27,Pediatria,2023-07-24 19:05:00,2023-07-24 19:05:00,Realizada,0.0,2
62,85,34,23,75,Ortopedia,2023-04-07 15:40:00,2023-04-07 15:40:00,Realizada,0.0,1
